In [1]:
import pandas as pd
import numpy as np
import os
import datetime
import gc
import glob

In [2]:
def recursive_file_gen(root_folder):
    for root, dirs, files in os.walk(root_folder):
        for file in files:
            yield os.path.join(root, file)

In [3]:
Q2_start="2019-05-05"
Q2_end="2019-08-03"

daily_item_files_2019Q2=list(recursive_file_gen("/home/jian/BigLots/2019_by_weeks/"))
daily_item_files_2019Q2=[x for x in daily_item_files_2019Q2 if x[-4:]==".txt" and "daily" in x.lower()]
daily_item_files_2019Q2=[x for x in daily_item_files_2019Q2 if x.split("s/MediaStorm_")[1][:10]>=Q2_start]
daily_item_files_2019Q2=[x for x in daily_item_files_2019Q2 if x.split("s/MediaStorm_")[1][:10]<=Q2_end]

print(len(daily_item_files_2019Q2))

daily_item_files_2019Q2

13


['/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-05-11/MediaStormDailySales20190514-121223-405.txt',
 '/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-05-18/MediaStormDailySales20190521-111232-391.txt',
 '/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-05-25/MediaStormDailySales20190528-111211-863.txt',
 '/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-06-01/MediaStormDailySales20190604-111936-950.txt',
 '/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-06-08/MediaStormDailySales20190611-122013-442.txt',
 '/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-06-15/MediaStormDailySales20190618-112500-846.txt',
 '/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-06-22/MediaStormDailySales20190625-111927-257.txt',
 '/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-06-29/MediaStormDailySales20190702-113523-928.txt',
 '/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-07-06/MediaStormDailySales20190709-115715-409.txt',
 '/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-07-13/MediaStormDailyS

In [4]:
df_output_Q2_sales=pd.DataFrame()
for file in daily_item_files_2019Q2:
    week_end_dt=file.split("s/MediaStorm_")[1][:10]
    df=pd.read_table(file,dtype=str,sep="|",usecols=['location_id','transaction_dt', 'transaction_id',
                                                              'customer_id_hashed', 'item_transaction_amt'])
    df['item_transaction_amt']=df['item_transaction_amt'].astype(float)
    # Rewards
    df_rewards=df[pd.notnull(df['customer_id_hashed'])]
    df_rewards_sales=df_rewards.groupby(["location_id"])['item_transaction_amt'].sum().to_frame().reset_index()
    df_rewards_sales=df_rewards_sales.rename(columns={"item_transaction_amt":"sales"})
    df_rewards_trans=df_rewards[['location_id','transaction_dt', 'transaction_id','customer_id_hashed']].drop_duplicates()
    df_rewards_trans['trans']=1
    df_rewards_trans=df_rewards_trans.groupby(["location_id"])['trans'].sum().to_frame().reset_index()
    
    df_rewards=pd.merge(df_rewards_trans,df_rewards_sales,on="location_id",how="outer")
    df_rewards['rewards_label']="rewards"
    
    # Non Rewards
    df_non_rewards=df[pd.isnull(df['customer_id_hashed'])]
    df_non_rewards_sales=df_non_rewards.groupby(["location_id"])['item_transaction_amt'].sum().to_frame().reset_index()
    df_non_rewards_sales=df_non_rewards_sales.rename(columns={"item_transaction_amt":"sales"})
    df_non_rewards_trans=df_non_rewards[['location_id','transaction_dt', 'transaction_id']].drop_duplicates()
    df_non_rewards_trans['trans']=1
    df_non_rewards_trans=df_non_rewards_trans.groupby(["location_id"])['trans'].sum().to_frame().reset_index()  
    
    df_non_rewards=pd.merge(df_non_rewards_sales,df_non_rewards_trans,on="location_id",how="outer")
    df_non_rewards['rewards_label']="non_rewards"
    
    df=df_rewards.append(df_non_rewards)
    df['store_type']=np.where(df['location_id']=="6990","online","instre")
    df=df.groupby(["rewards_label","store_type"])['sales','trans'].sum().reset_index()
    df['week_end_dt']=week_end_dt
    
    df_output_Q2_sales=df_output_Q2_sales.append(df)
    print(datetime.datetime.now(),file)

2019-10-18 18:07:31.274347 /home/jian/BigLots/2019_by_weeks/MediaStorm_2019-05-11/MediaStormDailySales20190514-121223-405.txt
2019-10-18 18:08:05.087358 /home/jian/BigLots/2019_by_weeks/MediaStorm_2019-05-18/MediaStormDailySales20190521-111232-391.txt
2019-10-18 18:08:42.116088 /home/jian/BigLots/2019_by_weeks/MediaStorm_2019-05-25/MediaStormDailySales20190528-111211-863.txt
2019-10-18 18:09:17.725537 /home/jian/BigLots/2019_by_weeks/MediaStorm_2019-06-01/MediaStormDailySales20190604-111936-950.txt
2019-10-18 18:09:52.486045 /home/jian/BigLots/2019_by_weeks/MediaStorm_2019-06-08/MediaStormDailySales20190611-122013-442.txt
2019-10-18 18:10:27.339252 /home/jian/BigLots/2019_by_weeks/MediaStorm_2019-06-15/MediaStormDailySales20190618-112500-846.txt
2019-10-18 18:10:58.822838 /home/jian/BigLots/2019_by_weeks/MediaStorm_2019-06-22/MediaStormDailySales20190625-111927-257.txt
2019-10-18 18:11:33.145188 /home/jian/BigLots/2019_by_weeks/MediaStorm_2019-06-29/MediaStormDailySales20190702-113523-

In [5]:
writer=pd.ExcelWriter("./BL_Q2_total_sales_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
df_output_Q2_sales.to_excel(writer,"Q2_total_sales",index=False)
writer.save()

In [6]:
df_output_Q2_sales.shape

(52, 5)

In [13]:
df_non_rewards_sales['sales'].sum()

32784381.150000036

In [12]:
df_non_rewards_trans['trans'].sum()

1301872